# CHEAT SHEET - Selección de Variables y Validación de Modelos

## 📚 ÍNDICE

### 1. Configuración Inicial
- [1.1. Librerías Necesarias](#11-librerías-necesarias)
- [1.2. Limpieza de Entorno](#12-limpieza-de-entorno)
- [1.3. Carga de Datos](#13-carga-de-datos)

### 2. Preparación de Datos
- [2.1. Separación X e Y](#21-separación-x-e-y)
- [2.2. Train-Test Split](#22-train-test-split)

### 3. Modelo Lineal Completo
- [3.1. Ajuste del Modelo](#31-ajuste-del-modelo)
- [3.2. Evaluación Train vs Test](#32-evaluación-train-vs-test)

### 4. Selección Secuencial de Variables
- [4.1. Forward Selection](#41-forward-selection)
- [4.2. Visualización de Errores](#42-visualización-de-errores)

### 5. Criterios de Información
- [5.1. AIC (Akaike Information Criterion)](#51-aic-akaike-information-criterion)
- [5.2. BIC (Bayesian Information Criterion)](#52-bic-bayesian-information-criterion)

### 6. Validación Cruzada (Cross-Validation)
- [6.1. K-Fold Cross-Validation](#61-k-fold-cross-validation)
- [6.2. Selección del Modelo Óptimo](#62-selección-del-modelo-óptimo)

### 7. Tips y Mejores Prácticas

---

## 1. CONFIGURACIÓN INICIAL

### 1.1. Librerías Necesarias

In [ ]:
# ============================================
# LIBRERÍAS ESENCIALES PARA SELECCIÓN DE MODELOS
# ============================================

import os                          # Manejo de directorios y archivos
import pandas as pd                # Manipulación de datos tabulares
import numpy as np                 # Operaciones numéricas y arrays
import random                      # Generación de números aleatorios
import pylab as pl                 # Visualización (wrapper de matplotlib)

# Statsmodels: modelos estadísticos clásicos (OLS, AIC, BIC)
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

# Scikit-learn: machine learning y validación
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

print("✅ Librerías cargadas correctamente")

### 1.2. Limpieza de Entorno

**Propósito:** Asegurar un entorno limpio sin variables previas que puedan interferir.

In [ ]:
# ============================================
# LIMPIEZA DE ENTORNO (OPCIONAL EN JUPYTER)
# ============================================
# Útil cuando trabajas en sesiones largas o reutilizas notebooks

from IPython import get_ipython

# %reset -f: elimina todas las variables del workspace
# %clear: limpia el output de celdas anteriores
get_ipython().run_line_magic('reset', '-f')
get_ipython().run_line_magic('clear', '-f')

print("✅ Entorno limpio y listo")

# ALTERNATIVA sin IPython (para scripts):
# No es necesario limpiar, Python gestiona memoria automáticamente

### 1.3. Carga de Datos

**Dataset:** `data_mvn.txt` - Datos de distribución multivariante normal  
**Formato:** Delimitado por tabulaciones (`\t`)

In [ ]:
# ============================================
# CARGA DE DATOS
# ============================================

# Verificar directorio de trabajo actual
print(f"Directorio actual: {os.getcwd()}")

# Cargar datos desde archivo delimitado por tabulaciones
df = pd.read_csv('data_mvn.txt', delimiter='\t')

# Exploración inicial del dataset
print(f"\nShape del dataset: {df.shape}")  # (filas, columnas)
print(f"\nColumnas: {df.columns.tolist()}")
print(f"\nPrimeras filas:\n{df.head()}")
print(f"\nInformación del dataset:")
print(df.info())
print(f"\nEstadísticas descriptivas:\n{df.describe()}")

# Verificar valores faltantes
print(f"\nValores nulos por columna:\n{df.isnull().sum()}")

## 2. PREPARACIÓN DE DATOS

### 2.1. Separación X e Y

**Concepto clave:**
- **X (Features):** Variables predictoras/independientes
- **Y (Target):** Variable objetivo/dependiente a predecir
- **Intercepto:** Término constante β₀ en la regresión Y = β₀ + β₁X₁ + ... + βₚXₚ

In [ ]:
# ============================================
# PREPARACIÓN DE VARIABLES X e Y
# ============================================

# Separar features (X) de target (Y)
X = df.drop(columns=['Y'], axis=1).values  # Todas las columnas excepto 'Y'
Y = df['Y'].values                          # Solo columna 'Y'

# Dimensiones del problema
n, p_original = df.shape  # n = observaciones, p = variables totales
p = p_original - 1        # p = número de predictores (sin contar Y)

print(f"Número de observaciones (n): {n}")
print(f"Número de predictores (p): {p}")
print(f"Shape de X: {X.shape}")  # (n, p)
print(f"Shape de Y: {Y.shape}")  # (n,)

# ============================================
# AÑADIR INTERCEPTO (TÉRMINO CONSTANTE)
# ============================================
# statsmodels NO añade intercepto automáticamente (sklearn SÍ)
# sm.add_constant() añade columna de unos al inicio

X = sm.add_constant(X)
print(f"\nShape de X con intercepto: {X.shape}")  # (n, p+1)

# Ahora X tiene estructura: [1, X1, X2, ..., Xp]
# donde la primera columna son unos (intercepto)

print(f"\nPrimeras filas de X (con intercepto):\n{X[:3, :]}")

### 2.2. Train-Test Split

**División estratégica de datos:**
- **Training set (50%):** Para entrenar/ajustar el modelo
- **Test set (50%):** Para evaluar rendimiento en datos no vistos
- **Random seed:** Garantiza reproducibilidad de resultados

In [ ]:
# ============================================
# DIVISIÓN TRAIN-TEST
# ============================================

# Fijar semilla para reproducibilidad
# Cualquier ejecución con seed=100 dará mismos resultados
random.seed(100)
np.random.seed(100)  # También fijar para numpy

# División 50-50 (puede ajustarse según necesidad)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=0.5,      # 50% para test
    random_state=100    # Semilla para reproducibilidad
)

print(f"Tamaño train: {X_train.shape[0]} observaciones")
print(f"Tamaño test:  {X_test.shape[0]} observaciones")
print(f"\nProporción train/test: {X_train.shape[0]/n:.1%} / {X_test.shape[0]/n:.1%}")

# IMPORTANTE: 
# - Entrenar SOLO con datos de train
# - Evaluar rendimiento final con datos de test
# - Nunca usar test durante entrenamiento (data leakage)

## 3. MODELO LINEAL COMPLETO

### 3.1. Ajuste del Modelo

**Modelo:** Y = β₀ + β₁X₁ + β₂X₂ + ... + βₚXₚ + ε  
**Método:** OLS (Ordinary Least Squares) - Mínimos Cuadrados Ordinarios  
**Objetivo:** Minimizar Σ(Y - Ŷ)²

In [ ]:
# ============================================
# MODELO LINEAL CON TODAS LAS VARIABLES
# ============================================

# Ajustar modelo OLS con statsmodels
model_fit = sm.OLS(Y_train, X_train).fit()

# Predicciones en conjunto de entrenamiento
ypred_train = model_fit.predict(X_train)

# Calcular RMSE (Root Mean Squared Error) en train
rmse_train = rmse(Y_train, ypred_train)
print(f"RMSE Training: {rmse_train:.4f}")

# Ver resumen completo del modelo
print("\n" + "="*70)
print("RESUMEN DEL MODELO COMPLETO")
print("="*70)
print(model_fit.summary())

# Información clave del resumen:
# - R-squared: proporción de varianza explicada (0-1, mayor es mejor)
# - Adj. R-squared: R² ajustado por número de variables
# - F-statistic: significancia global del modelo
# - Coef: valores de los coeficientes β
# - P>|t|: p-valores (< 0.05 indica significancia)
# - AIC/BIC: criterios de información (menor es mejor)

### 3.2. Evaluación Train vs Test

**Conceptos fundamentales:**
- **Overfitting:** Modelo aprende ruido del train → mal rendimiento en test
- **Underfitting:** Modelo demasiado simple → mal rendimiento en train y test
- **Balance ideal:** RMSE similar en train y test

In [ ]:
# ============================================
# EVALUACIÓN EN TEST SET
# ============================================

# Predicciones en conjunto de test (datos NO vistos durante entrenamiento)
ypred_test = model_fit.predict(X_test)

# Calcular RMSE en test
rmse_test = rmse(Y_test, ypred_test)

# Comparar rendimiento train vs test
print("\n" + "="*70)
print("COMPARACIÓN TRAIN vs TEST")
print("="*70)
print(f"RMSE Training: {rmse_train:.4f}")
print(f"RMSE Test:     {rmse_test:.4f}")
print(f"Diferencia:    {abs(rmse_test - rmse_train):.4f}")
print(f"Ratio Test/Train: {rmse_test/rmse_train:.2f}")

# INTERPRETACIÓN:
if rmse_test > rmse_train * 1.2:
    print("\n⚠️  OVERFITTING: RMSE test >> RMSE train")
    print("   → Modelo se ajusta demasiado a datos de entrenamiento")
    print("   → Considerar regularización o reducir variables")
elif rmse_test < rmse_train * 0.8:
    print("\n⚠️  INUSUAL: RMSE test < RMSE train")
    print("   → Puede indicar datos de test más fáciles de predecir")
else:
    print("\n✅ BALANCE ADECUADO: RMSE similar en train y test")
    print("   → Modelo generaliza bien a datos nuevos")

# Visualización de predicciones
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico Train
ax1.scatter(Y_train, ypred_train, alpha=0.6, edgecolors='k', linewidths=0.5)
ax1.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 
         'r--', lw=2, label='Predicción Perfecta')
ax1.set_xlabel('Y Real (Train)')
ax1.set_ylabel('Y Predicho')
ax1.set_title(f'Train Set (RMSE={rmse_train:.4f})')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico Test
ax2.scatter(Y_test, ypred_test, alpha=0.6, edgecolors='k', linewidths=0.5, color='orange')
ax2.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 
         'r--', lw=2, label='Predicción Perfecta')
ax2.set_xlabel('Y Real (Test)')
ax2.set_ylabel('Y Predicho')
ax2.set_title(f'Test Set (RMSE={rmse_test:.4f})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. SELECCIÓN SECUENCIAL DE VARIABLES

### 4.1. Forward Selection

**Estrategia:** Añadir variables una por una en orden secuencial  
**Iteración k:** Usar primeras k variables (intercepto + X₁ + X₂ + ... + Xₖ)  
**Objetivo:** Identificar cuántas variables son necesarias

In [ ]:
# ============================================
# FORWARD SELECTION: MODELOS SECUENCIALES
# ============================================

# Inicializar arrays para almacenar RMSE
rmse_train_array = np.zeros(p+1)  # +1 para incluir modelo solo con intercepto
rmse_test_array = np.zeros(p+1)
length = range(0, p+1)  # Número de variables: 0, 1, 2, ..., p

print("Ajustando modelos con k=1, 2, ..., p variables...\n")

# Iterar sobre número de variables
k = 1
while k <= (p+1):
    # Seleccionar primeras k columnas (intercepto + k-1 variables)
    Xk_train = X_train[:, :k]
    Xk_test = X_test[:, :k]
    
    # Ajustar modelo con k variables
    model_L = sm.OLS(Y_train, Xk_train)
    model_fit = model_L.fit()
    
    # Predicciones y RMSE en train
    ypred_train = model_fit.predict(Xk_train)
    rmse_train_array[k-1] = rmse(Y_train, ypred_train)
    
    # Predicciones y RMSE en test
    ypred_test = model_fit.predict(Xk_test)
    rmse_test_array[k-1] = rmse(Y_test, ypred_test)
    
    # Mostrar progreso cada 5 iteraciones
    if k % 5 == 0 or k <= 3:
        print(f"k={k:2d} variables: RMSE_train={rmse_train_array[k-1]:.4f}, "
              f"RMSE_test={rmse_test_array[k-1]:.4f}")
    
    k += 1

print("\n✅ Forward selection completado")

# Encontrar mejor k según test error
best_k = np.argmin(rmse_test_array) + 1
print(f"\nMejor número de variables según test error: k={best_k}")
print(f"RMSE test mínimo: {rmse_test_array[best_k-1]:.4f}")

### 4.2. Visualización de Errores

**Curva de aprendizaje:**
- **Train error:** Siempre decrece (más variables = mejor ajuste)
- **Test error:** Forma de U (óptimo en el mínimo)
- **Gap train-test:** Indica overfitting si crece mucho

In [ ]:
# ============================================
# VISUALIZACIÓN: CURVAS DE APRENDIZAJE
# ============================================

plt.figure(figsize=(12, 6))

# Plot de errores
plt.plot(length, rmse_train_array, 'b-o', label='Training Error', linewidth=2, markersize=6)
plt.plot(length, rmse_test_array, 'r-s', label='Test Error', linewidth=2, markersize=6)

# Marcar punto óptimo
plt.axvline(x=best_k, color='green', linestyle='--', linewidth=2, 
           label=f'Óptimo (k={best_k})')
plt.scatter([best_k], [rmse_test_array[best_k-1]], 
           color='green', s=200, zorder=5, marker='*')

plt.xlabel('Número de Variables (k)', fontsize=12)
plt.ylabel('RMSE', fontsize=12)
plt.title('Forward Selection: Train vs Test Error', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# INTERPRETACIÓN:
print("\n" + "="*70)
print("INTERPRETACIÓN DE LA CURVA")
print("="*70)
print("\n📉 TRAIN ERROR (azul):")
print("   - Decrece monotónicamente al añadir variables")
print("   - Más variables → mejor ajuste a datos de entrenamiento")
print("   - NO es buen criterio para selección (puede overfit)")

print("\n📈 TEST ERROR (rojo):")
print("   - Forma de U: decrece y luego aumenta")
print("   - Mínimo indica número óptimo de variables")
print("   - Después del mínimo: overfitting")

print("\n🎯 PUNTO ÓPTIMO (verde):")
print(f"   - k={best_k} variables minimiza error en test")
print(f"   - Balance entre bias (underfitting) y variance (overfitting)")

## 5. CRITERIOS DE INFORMACIÓN

### 5.1. AIC (Akaike Information Criterion)

**Fórmula:** AIC = 2k - 2ln(L)  
**Donde:**
- k = número de parámetros
- L = likelihood (verosimilitud del modelo)

**Interpretación:** Menor AIC = mejor modelo  
**Penalización:** Leve por número de parámetros

### 5.2. BIC (Bayesian Information Criterion)

**Fórmula:** BIC = k·ln(n) - 2ln(L)  
**Donde:**
- k = número de parámetros
- n = número de observaciones
- L = likelihood

**Interpretación:** Menor BIC = mejor modelo  
**Penalización:** Más fuerte que AIC (prefiere modelos más simples)

In [ ]:
# ============================================
# CRITERIOS DE INFORMACIÓN: AIC y BIC
# ============================================

# Inicializar arrays
AIC = np.zeros(p+1)
BIC = np.zeros(p+1)

print("Calculando AIC y BIC para k=1, 2, ..., p variables...\n")

# IMPORTANTE: Usar TODOS los datos (X, Y) no solo train
# AIC/BIC son criterios para selección de modelo, no para evaluación
k = 1
while k <= (p+1):
    # Seleccionar primeras k variables
    Xk = X[:, :k]
    
    # Ajustar modelo
    model_L = sm.OLS(Y, Xk)
    model_fit = model_L.fit()
    
    # Extraer AIC y BIC
    AIC[k-1] = model_fit.aic
    BIC[k-1] = model_fit.bic
    
    if k % 5 == 0 or k <= 3:
        print(f"k={k:2d}: AIC={AIC[k-1]:.2f}, BIC={BIC[k-1]:.2f}")
    
    k += 1

# Encontrar óptimos
best_k_aic = np.argmin(AIC) + 1
best_k_bic = np.argmin(BIC) + 1

print(f"\n✅ Mejor k según AIC: {best_k_aic} (AIC={AIC[best_k_aic-1]:.2f})")
print(f"✅ Mejor k según BIC: {best_k_bic} (BIC={BIC[best_k_bic-1]:.2f})")

# Visualización
plt.figure(figsize=(12, 6))

plt.plot(length, AIC, 'b-o', label='AIC', linewidth=2, markersize=6)
plt.plot(length, BIC, 'r-s', label='BIC', linewidth=2, markersize=6)

# Marcar óptimos
plt.axvline(x=best_k_aic, color='blue', linestyle='--', alpha=0.5, 
           label=f'Óptimo AIC (k={best_k_aic})')
plt.axvline(x=best_k_bic, color='red', linestyle='--', alpha=0.5, 
           label=f'Óptimo BIC (k={best_k_bic})')

plt.xlabel('Número de Variables (k)', fontsize=12)
plt.ylabel('Valor del Criterio', fontsize=12)
plt.title('Criterios de Información: AIC vs BIC', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# COMPARACIÓN
print("\n" + "="*70)
print("COMPARACIÓN AIC vs BIC")
print("="*70)
print("\n📊 AIC (Akaike Information Criterion):")
print(f"   - Óptimo: k={best_k_aic} variables")
print("   - Penalización leve por complejidad")
print("   - Prefiere modelos más complejos que BIC")
print("   - Mejor para predicción")

print("\n📊 BIC (Bayesian Information Criterion):")
print(f"   - Óptimo: k={best_k_bic} variables")
print("   - Penalización fuerte por complejidad")
print("   - Prefiere modelos más simples (parsimonia)")
print("   - Mejor para interpretación")

if best_k_bic < best_k_aic:
    print("\n💡 BIC selecciona modelo más simple (usual)")
elif best_k_bic > best_k_aic:
    print("\n💡 AIC selecciona modelo más simple (inusual)")
else:
    print("\n💡 AIC y BIC coinciden en el modelo óptimo")

## 6. VALIDACIÓN CRUZADA (CROSS-VALIDATION)

### 6.1. K-Fold Cross-Validation

**Concepto:** Dividir datos en K folds (subconjuntos)  
**Proceso:**
1. Entrenar con K-1 folds
2. Validar con el fold restante
3. Repetir K veces (cada fold como validación una vez)
4. Promediar resultados

**Ventajas:**
- Usa todos los datos para train y validación
- Estimación más robusta del error
- Reduce varianza de la estimación

In [ ]:
# ============================================
# CROSS-VALIDATION: 5-FOLD CV
# ============================================

# Inicializar
CV = np.zeros(p+1)
lm = LinearRegression()  # Usar sklearn para compatibilidad con cross_val_score

print("Ejecutando 5-Fold Cross-Validation...\n")
print("Esto puede tardar un momento...\n")

k = 1
while k <= (p+1):
    # Seleccionar primeras k variables
    Xk = X_train[:, :k]
    
    # Ejecutar cross-validation con 5 folds
    # scoring='neg_root_mean_squared_error': devuelve -RMSE (negativo para maximizar)
    scores = -cross_val_score(
        lm, Xk, Y_train,
        scoring='neg_root_mean_squared_error',
        cv=5,              # 5 folds
        n_jobs=-1          # paralelizar
    )
    
    # Promedio de RMSE en los 5 folds
    CV[k-1] = np.mean(scores)
    
    if k % 5 == 0 or k <= 3:
        print(f"k={k:2d}: CV-RMSE={CV[k-1]:.4f} (std={np.std(scores):.4f})")
    
    k += 1

print("\n✅ Cross-validation completado")

# Encontrar mejor k según CV
best_k_cv = np.argmin(CV) + 1
print(f"\nMejor k según Cross-Validation: {best_k_cv}")
print(f"CV-RMSE mínimo: {CV[best_k_cv-1]:.4f}")

# Visualización comparativa
plt.figure(figsize=(14, 6))

plt.plot(length, rmse_train_array, 'b-o', label='Training Error', 
         linewidth=2, markersize=6, alpha=0.7)
plt.plot(length, rmse_test_array, 'r-s', label='Test Error', 
         linewidth=2, markersize=6, alpha=0.7)
plt.plot(length, CV, 'g-^', label='CV Error (5-Fold)', 
         linewidth=2, markersize=6)

# Marcar óptimo CV
plt.axvline(x=best_k_cv, color='green', linestyle='--', linewidth=2, 
           label=f'Óptimo CV (k={best_k_cv})')
plt.scatter([best_k_cv], [CV[best_k_cv-1]], 
           color='green', s=200, zorder=5, marker='*')

plt.xlabel('Número de Variables (k)', fontsize=12)
plt.ylabel('RMSE', fontsize=12)
plt.title('Comparación: Train, Test y Cross-Validation Error', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("INTERPRETACIÓN DE CROSS-VALIDATION")
print("="*70)
print("\n✅ VENTAJAS:")
print("   - Usa todos los datos de train eficientemente")
print("   - Estimación más robusta que single train-test split")
print("   - Reduce varianza de la estimación del error")
print("   - Mejor generalización a datos nuevos")

print("\n💡 CV Error suele estar entre Train y Test Error")
print(f"   Train: {rmse_train_array[best_k_cv-1]:.4f}")
print(f"   CV:    {CV[best_k_cv-1]:.4f}")
print(f"   Test:  {rmse_test_array[best_k_cv-1]:.4f}")

### 6.2. Selección del Modelo Óptimo

**Modelo final:** Usar k óptimo determinado por CV  
**Entrenamiento final:** Con todos los datos de train  
**Evaluación final:** Con datos de test

In [ ]:
# ============================================
# MODELO FINAL CON K ÓPTIMO
# ============================================

# Extraer k óptimo de CV
min_value = min(CV)
min_index = np.where(CV == min_value)
K = np.ndarray.item(min_index[0]) + 1  # +1 porque índices empiezan en 0

print(f"Número óptimo de variables: K = {K}")
print(f"CV-RMSE óptimo: {min_value:.4f}\n")

# Preparar datos con K variables
X_train_k = X_train[:, :K]
X_test_k = X_test[:, :K]

# Entrenar modelo final con K variables
model_L = sm.OLS(Y_train, X_train_k)
model_fit_final = model_L.fit()

# Predicciones
ypred_train_final = model_fit_final.predict(X_train_k)
ypred_test_final = model_fit_final.predict(X_test_k)

# Calcular RMSE
rmse_train_final = rmse(Y_train, ypred_train_final)
rmse_test_final = rmse(Y_test, ypred_test_final)

# Resultados finales
print("="*70)
print("RESULTADOS DEL MODELO FINAL OPTIMIZADO")
print("="*70)
print(f"\nNúmero de variables seleccionadas: {K}")
print(f"RMSE Training:   {rmse_train_final:.4f}")
print(f"RMSE Test:       {rmse_test_final:.4f}")
print(f"RMSE CV (5-fold): {CV[K-1]:.4f}")

# Comparación con modelo completo
print(f"\n📊 COMPARACIÓN CON MODELO COMPLETO (p={p} variables):")
print(f"   Modelo completo - RMSE Test: {rmse_test:.4f}")
print(f"   Modelo óptimo   - RMSE Test: {rmse_test_final:.4f}")
print(f"   Mejora: {(rmse_test - rmse_test_final)/rmse_test * 100:.2f}%")
print(f"   Reducción de variables: {p} → {K} ({(p-K)/p*100:.1f}% menos)")

# Resumen del modelo
print(f"\n{model_fit_final.summary()}")

# Visualización final
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train
axes[0].scatter(Y_train, ypred_train_final, alpha=0.6, edgecolors='k', linewidths=0.5)
axes[0].plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 
            'r--', lw=2, label='Predicción Perfecta')
axes[0].set_xlabel('Y Real (Train)', fontsize=11)
axes[0].set_ylabel('Y Predicho', fontsize=11)
axes[0].set_title(f'Modelo Final - Train (K={K}, RMSE={rmse_train_final:.4f})', 
                 fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test
axes[1].scatter(Y_test, ypred_test_final, alpha=0.6, edgecolors='k', 
               linewidths=0.5, color='orange')
axes[1].plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 
            'r--', lw=2, label='Predicción Perfecta')
axes[1].set_xlabel('Y Real (Test)', fontsize=11)
axes[1].set_ylabel('Y Predicho', fontsize=11)
axes[1].set_title(f'Modelo Final - Test (K={K}, RMSE={rmse_test_final:.4f})', 
                 fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. TIPS Y MEJORES PRÁCTICAS

### 📊 Resumen de Métodos de Selección

| Método | Ventajas | Desventajas | Cuándo usar |
|--------|----------|-------------|-------------|
| **Test Error** | Simple, directo | Depende de split específico | Cuando hay suficientes datos |
| **AIC** | Rápido, matemático | Puede sobreajustar | Predicción, muchos datos |
| **BIC** | Parsimonia, robusto | Puede subajustar | Interpretación, explicación |
| **Cross-Validation** | Robusto, confiable | Computacionalmente costoso | Datos limitados, mejor práctica |

### ✅ HACER (Best Practices):

1. **Siempre dividir datos:** Train-Test split ANTES de cualquier análisis
2. **Nunca usar test en entrenamiento:** Test solo para evaluación final
3. **Fijar random seed:** Para reproducibilidad (`random.seed(100)`)
4. **Usar CV para selección:** Más robusto que single split
5. **Añadir intercepto con statsmodels:** `sm.add_constant(X)`
6. **Comparar múltiples métodos:** AIC, BIC, CV pueden no coincidir
7. **Visualizar curvas de aprendizaje:** Identificar overfitting/underfitting
8. **Revisar significancia de variables:** P-valores en resumen del modelo
9. **Verificar supuestos de regresión:** Residuos normales, homocedasticidad
10. **Documentar decisiones:** Justificar elección del modelo final

### ❌ EVITAR (Common Mistakes):

1. ❌ Usar test error para selección de modelo → Data leakage
2. ❌ No fijar seed → Resultados no reproducibles
3. ❌ Olvidar intercepto → Modelo sesgado
4. ❌ Confiar solo en R² → No indica generalización
5. ❌ Elegir modelo más complejo automáticamente → Overfitting
6. ❌ Ignorar p-valores → Variables no significativas
7. ❌ No validar supuestos → Conclusiones erróneas
8. ❌ Comparar RMSE de datasets diferentes → No comparables

### 🎯 Criterios de Decisión:

**Si BIC y CV coinciden → USAR ESE MODELO** (más confiable)

**Si difieren:**
- BIC < CV: Modelo más simple (mejor interpretación)
- CV < BIC: Modelo más complejo (mejor predicción)
- Decisión depende del objetivo del análisis

**Objetivo Predicción:** Usar CV (generalización)

**Objetivo Interpretación:** Usar BIC (parsimonia)

### 🔍 Checklist de Validación:

- [ ] Train-test split realizado correctamente
- [ ] Seed fijado para reproducibilidad
- [ ] Intercepto añadido si usa statsmodels
- [ ] Forward selection ejecutado
- [ ] AIC/BIC calculados
- [ ] Cross-validation ejecutado
- [ ] Modelo final entrenado con k óptimo
- [ ] RMSE train, test y CV reportados
- [ ] Gráficos de predicción vs real generados
- [ ] Residuos verificados (opcional pero recomendado)

### 📚 Recursos Adicionales:

**Statsmodels:** https://www.statsmodels.org/

**Scikit-learn:** https://scikit-learn.org/

**Cross-Validation:** https://scikit-learn.org/stable/modules/cross_validation.html

**Model Selection:** https://scikit-learn.org/stable/model_selection.html

---

## 🎓 CONCLUSIÓN

Este notebook cubre el **flujo completo de selección de variables** en regresión lineal:

1. ✅ Preparación y split de datos
2. ✅ Modelo completo como baseline
3. ✅ Forward selection para explorar complejidad
4. ✅ AIC/BIC para selección matemática
5. ✅ Cross-validation para selección robusta
6. ✅ Modelo final optimizado

**Recuerda:** No existe un único "mejor" método. La elección depende de:
- Tamaño del dataset
- Objetivo (predicción vs interpretación)
- Recursos computacionales
- Contexto del problema

**Best Practice:** Usar múltiples métodos y comparar resultados.

---

*Cheat sheet creado con fines educativos - Data Science Master*